# Containers n stuff

Oh how lucky you are!

Back in my day you actually had to set up the environment
with installations and scripts.

Swapping OS was a nightmare.

Fortunately containers solve this problem.

## Container engines
Container engines allow users to run a prebuilt enviroment in a sandbox.
The same environment on any machine, making deployment A LOT easier.

### Docker
Docker was the first container technology to actually take off.
Unfortunately it has a major security flaw, you have to run a daemon
as root. It's also not OCI compliant as far as I know.

### Podman
Podman aims to solve these problems while also adopting as much of 
a common API with Docker as possible. It even has a shim for the docker command.

Unfortunately it runs only under linux.

## Installation
Because it is simple to install and run on non-server enviroments we will be using Docker.

* Windows https://docs.docker.com/docker-for-windows/install/
* MacOS https://docs.docker.com/docker-for-mac/install/
* Linux https://docs.docker.com/engine/install/ubuntu/

Probably `sudo apt-get install docker-ce docker-ce-cli containerd.io`

## Images
Images allow us to create a snapshot of a system that can be run anywhere with a container engine.

Each image can be ran as a container, in an isolated enviroment.

But ussually you do not want a 100% isolation, rather to export a port and maybe some disk.

## Let's play
Let's make a mariadb database


First lets kill all containers, just to be sure we are not running anything

In [1]:
# docker kill $(docker ps -q)
# docker system prune -f > /dev/null

In [1]:
docker run -p 127.0.0.1:3306:3306  --rm --name mariadb -e MARIADB_ROOT_PASSWORD=pass -d mariadb:latest

Unable to find image 'mariadb:latest' locally
latest: Pulling from library/mariadb

3491a907: Pulling fs layer 
1312ef6f: Pulling fs layer 
50ddb7d0: Pulling fs layer 
2e2ff778: Pulling fs layer 
a7dc2af9: Pulling fs layer 
f4f4774a: Pulling fs layer 
9f940bd2: Pulling fs layer 
723e99d8: Pulling fs layer 
bf0f673e: Pulling fs layer 
93730ad6: Pulling fs layer 
25a0d734: Pulling fs layer 
Digest: sha256:0c3c560359a6da112134a52122aa9b78fec5f9dd292a01ee7954de450f25f0c1
Status: Downloaded newer image for mariadb:latest
fc660432898756e865900e99fb8e402ff8368e24d3da75d73047edb478a7ff93


In [3]:
# To connect from a locally instaled mysql/mariadb cli
# mariadb -uroot -ppass -h127.0.0.1
# mysql -uroot -ppass -h127.0.0.1

In [2]:
# What if we don't have one? Docker to the rescue:
docker run -it --rm mariadb mysql -hmariadb -uroot -ppass
echo "died"

ERROR 2005 (HY000): Unknown MySQL server host 'mariadb' (-2)
(B(Bdied


As you can see, it's not working. Why is that?

Containers by default are isolated so you need to tell docker to connect them with a network.

Lets kill the container and make it again with a network (note that we can add it to a network even when running)

In [3]:
docker kill mariadb

mariadb


Lets check if there are any running containers

In [4]:
docker container ls

CONTAINER ID   IMAGE                           COMMAND                  CREATED          STATUS          PORTS                                       NAMES
2bf854ad15b3   mihai1voicescu/badass-jupyter   "docker-entrypoint.s…"   7 minutes ago    Up 7 minutes    0.0.0.0:8888->8888/tcp, :::8888->8888/tcp   loving_jemison
0f1b90f10c28   docker/getting-started:latest   "/docker-entrypoint.…"   37 minutes ago   Up 37 minutes   80/tcp                                      sad_faraday


Now lets create a network and add them to our network

In [5]:
docker network create awsome-network

53ba9ff0fd1a667085ad536ba43e0f2c21b026b876469e1cec888db3b523a021


In [6]:
docker run -p 127.0.0.1:3306:3306 --network awsome-network  --rm --name mariadb -e MARIADB_ROOT_PASSWORD=pass -d mariadb:latest

d303cdcbdd2d52755acbe63e09a3b2dcf2875bfed689ee77b0909a9bf89c5e1a


Now we can run this in a terminal (because it's interactive)

In [7]:
# docker run -it --network awsome-network --rm mariadb mysql -hmariadb -uroot -ppass

Now let's kill the mariadb instance

In [8]:
docker kill mariadb
docker system prune -f > /dev/null

mariadb
